## RAG Application with GPT-4o

In [1]:
from dotenv import load_dotenv

load_dotenv()

import os

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

### 1. Get a Data Loader

In [2]:
from langchain_community.document_loaders import WebBaseLoader

In [3]:
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
data = loader.load()
# data

### 2. Convert data to Vector Database

In [4]:
from langchain_objectbox.vectorstores import ObjectBox ## Vector Database
from langchain_openai import OpenAIEmbeddings

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(data)
# documents

In [6]:
from langchain_openai import OpenAIEmbeddings
vector = ObjectBox.from_documents(documents, OpenAIEmbeddings(), embedding_dimensions=768)
vector

### 3. Make a RAG Pipeline

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain import hub

In [8]:
llm = ChatOpenAI(model="gpt-4o")
prompt = hub.pull("rlm/rag-prompt")


In [9]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type_kwargs={"prompt" : prompt},
    retriever=vector.as_retriever(), 
)

In [10]:
question= "Explain what is LangSmith?"
result = qa_chain({"query": question})
# result

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [11]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(result["result"])

"I don't know."


In [12]:
question= "Explain Monitoring and A/B Testing in LangSmith?"
result = qa_chain({"query": question})
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(result["result"])

('In LangSmith, monitoring involves using tracing to debug and understand how '
 'your LLM application is performing, providing visibility and debugging '
 'information at each step of an LLM sequence. A/B testing is facilitated '
 'through a comparison view that lets you see results for different '
 'configurations side-by-side, helping you track and diagnose regressions in '
 'test scores across multiple revisions of your application. This allows for '
 'effective evaluation of different prompts, retrieval strategies, or model '
 'choices.')
